In [4]:
import numpy as np

In [5]:
# NTU RGB-D 120 data path

data_dir = '/data/zak/graph/ntu/train'

* The train directory contains video samples from 120 classes with around 114480 samples (some might be missing though)
* Each file name is in the following format:
    - `S013C003P037R001A004.skeleton.npy`
    - S013 stands for **Setup Number 13**
    - C003 stands for **Camera Number 03**
    - P037 stands for **Participant Number 037**
    - R001 stands for **Replication Number (001 or 002 only) << need to find out what this means**
    - A004 stands for **Action Class Number 004 (brush hair in this case)**

## Design data loader for the dataset

### A list of parameters that can be changed for the dataloader with their default values
- kp_shape = (25,3)
- seg_size = 61
- participant_list <= those who are in the train or validation or test set (a list of numbers/codes for the participants)
- data_path = '/data/zak/graph/ntu/train'

In [ ]:
class NTUDataset:
    
    def __init__(self, participant_list, data_path, kp_shape=(25, 3), seg_size=61):
        # Initialize all parameters for the model
        self.participant_list = participant_list
        self.kp_shape = kp_shape
        self.seg_size = seg_size
        self.data_path = data_path
        pass
    
    def __len__(self):
        # Number of samples in the dataset
        pass
    
    def __getitem(self, idx):
        # Return a particular item from the dataset
        pass
    
    # ----- Helper functions -----

# Utility Functions for Dataloader

In [21]:
import os
import random

# samples file_name = 'S018C001P042R002A120.skeleton.npy'
# P042 is the participant number

def split_train_val(data_path, val_pct=0.2):
    # Returns a random list of participants for the train and validation sets respectively
    samples = os.listdir(data_path)
    total_samples = len(samples)
    # Get all unique participant numbers
    all_participants = set()
    for sample in samples:
        part = sample.split('P')[1][:3]
        all_participants.add(part)
    total_participants = len(all_participants)
    all_participants = list(all_participants)
    
    # Split into train and val sets
    val_len = int(total_participants * val_pct)
    # Randomly shuffle the list
    random.shuffle(list(all_participants))
    train_participants = all_participants[val_len:]
    val_participants = all_participants[:val_len]

    print(f'Total Video Samples: {len(samples)} || Total Participants: {len(all_participants)} || Train Participants: {len(train_participants)} || Validation Participants: {len(val_participants)}')
    return train_participants, val_participants